In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt


In [28]:
orders = pd.read_csv('./data/Orders.csv')
returns = pd.read_csv('./data/Returns.csv')

In [30]:
## Replace $ and , in profit column and convert to numeric

orders['Profit'] = orders['Profit'].str.replace("$","")
orders['Profit'] = orders['Profit'].str.replace(",","")
orders['Profit'] = pd.to_numeric(orders['Profit'])

## Replace $ and , in sales column and convert to numeric

orders['Sales'] = orders['Sales'].str.replace("$","")
orders['Sales'] = orders['Sales'].str.replace(",","")
orders['Sales'] = pd.to_numeric(orders['Sales'])

<ipython-input-30-8a663bcd8bec>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  orders['Profit'] = orders['Profit'].str.replace("$","")
<ipython-input-30-8a663bcd8bec>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  orders['Sales'] = orders['Sales'].str.replace("$","")


In [ ]:
plt.figure(figsize=(5,5))
orders.plot.line(x = 'Order.Date', y = 'Sales', figsize = (15,5))

In [49]:
orders['Month'] = pd.DatetimeIndex(orders['Order.Date']).month

In [53]:
orders.plot.bar(x = 'Month',y= 'Sales')

KeyboardInterrupt: 